In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
data=pd.read_csv('NY property data.csv')

In [3]:
data.groupby(["BLOCK","ZIP"]).size()

BLOCK  ZIP    
1      10004.0      2
       10301.0    213
       11101.0      1
       11201.0      2
       11422.0      1
2      10004.0      2
       10301.0     10
3      10004.0      4
       10301.0      8
       11201.0      3
4      10004.0     53
       10301.0      3
5      10004.0      9
       10301.0     27
       11101.0      1
       11201.0      1
6      10004.0      2
       10301.0     19
       11101.0      3
       11201.0      5
7      10004.0     15
       10301.0     11
       11201.0      3
8      10004.0      6
       10301.0     14
9      10004.0      4
       10301.0      5
10     10004.0      8
       10301.0      1
11     10004.0     16
                 ... 
16302  11694.0     29
16303  11694.0     26
16304  11694.0     28
16305  11694.0     30
16306  11694.0     15
16307  11694.0      9
16308  11694.0      8
16309  11694.0     10
16310  11694.0     13
16311  11694.0     13
16312  11694.0     13
16313  11694.0     17
16314  11694.0      9
16315  11694.0   

In [101]:
data.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


# ZIP
Aggregate by block, select most common zip code. If still Na, aggregate by B

In [64]:
#import function to select most common value
from collections import Counter

def Most_Common(lst):
    data = Counter(lst)
    return data.most_common(1)[0][0]

In [136]:
#create new data frame only containing BLOCK and ZIP
zipBlock=data[["BLOCK","ZIP"]]

In [138]:
zipBlock["ZIP2"] = zipBlock.groupby("BLOCK").\
    transform(lambda x: x.fillna(Most_Common(x)))

In [32]:
# zipBlock["ZIP2"] = zipBlock.groupby("BLOCK").\
#     transform(lambda x: x.fillna(x.mode()))

In [68]:
zipBlock.count()

BLOCK    1070994
ZIP      1041104
ZIP2     1070434
dtype: int64

Aggregate by B

In [181]:
zipB=zipBlock.copy()
zipB["B"]=data["B"]
#zipB["TAXCLASS"]=data["TAXCLASS"]
zipB=zipB.drop(['BLOCK',"ZIP"],axis=1)

In [158]:
zipB.count()

B       1070994
ZIP5    1070434
dtype: int64

In [174]:
zipB["ZIP5"] = zipB.groupby("B").\
     transform(lambda x: x.fillna(Most_Common(x)))

In [180]:
#some how Most_Common doest work so use this new one instead
def most_common(lst):
    data = Counter(lst)
    return max(lst, key=data.get)

In [182]:
zipB["ZIP5"] = zipB.groupby("B").\
     transform(lambda x: x.fillna(most_common(x)))

In [184]:
zipB.count()

ZIP2    1070434
B       1070994
ZIP5    1070994
dtype: int64

In [183]:
zipB[zipB["ZIP5"].isnull()]

,ZIP2,B,ZIP5


In [185]:
zipB[zipB["ZIP5"]=="ZIP5"]

,ZIP2,B,ZIP5


In [186]:
zipB=zipB.drop(["ZIP2"],1)

# STORIES
Aggregate by block, select median stories. If still Na, aggregate by ZIP

In [187]:
storyBlock=data[["BLOCK","STORIES"]]

In [188]:
storyBlock.head()

,BLOCK,STORIES
0,1,NaN
1,1,NaN
2,2,3.0
3,2,2.0
4,3,1.0


In [189]:
storyBlock["STORIES2"] = storyBlock.groupby("BLOCK").\
    transform(lambda x: x.fillna(x.median()))

In [190]:
del storyBlock["STORIES"]
storyBlock.count()

BLOCK       1070994
STORIES2    1070180
dtype: int64

Aggregate by ZIP

In [191]:
storyZip=storyBlock.copy()
storyZip["ZIP5"]=zipB["ZIP5"]
del storyZip['BLOCK']

In [192]:
storyZip.head()

,STORIES2,ZIP5
0,1.0,10004
1,1.0,10004
2,3.0,10004
3,2.0,10004
4,1.0,10004


In [193]:
storyZip["STORIES2"] = storyZip.groupby("ZIP5").\
    transform(lambda x: x.fillna(x.median()))

In [194]:
storyZip.count()

STORIES2    1070994
ZIP5        1070994
dtype: int64

Convert to csv

In [195]:
storyZip.to_csv("storyZip2.csv")